<span type="title">模板方法模式和迭代器、组合模式</span> | <span type="update">2018-09-08</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章首先介绍了模板方法模式。模板方法通过在抽像层定义一组算法和服务流程，然后把这些步骤的实现推迟到子类进行，由子类选择实现或者重写抽象类的方法以提供自定义的服务。通过使用不同的关键字： final，protected，private，abstract，有效的指明了抽象类和子类的职责。此外，模板方法的算法可以有不同的分支，子类通过钩子进行控制。模板方法和工厂方法类似，都是交给子类，而策略模式则是使用组合，不过这三者都属于提供服务的目的（区别于上一张的适配器、装饰器、命令模式实现接口标准化的目的）。</p></span>

# 模板方法模式

## 模板方法定义

模板方法模式指的是，在抽象类中搭建框架，定义算法的步骤，然后把这些步骤的实现推迟到子类的设计模式。

模板方法的类图如下：

![](c5p1.png)

**模板方法的关键词和责任划分**

模板方法模式很像工厂方法模式，其区别在于，前者交给子类的是“方法”（算法的某个步骤），后者交给子类的是“对象”（然后对对象调用各种方法完成服务）。模板方法模式可以使用 abstract 限制以完全交给子类，也可以使用 protected 限制以实现钩子，由子类决定是否要实现，或者使用默认值，还可以使用 private 关键字限制以禁止子类实现，统一由父类完成。

**模板方法的钩子和算法结构逻辑**

此处的钩子不是指由子类决定实现或者不实现的方法（如果不实现，直接return null;），然后这些方法放在算法的序列语句中，如果为空则跳过。此处的钩子指的是，在算法中的分支情况。比如：

```java
process() {
    methodA();
    methodB();
    if (methodC()) {
        methodAdd();
    }
    methodD()
}
```

这样可以极大的扩展算法的结构，产生很多的分支，然后由抽象类维护这个具体提供服务的流程，而由子类决定其各个步骤的具体实现，以及选择什么步骤进行实现。

## 示例代码

```java
import java.io.BufferedReader;
import java.io.InputStreamReader;

public class DrinkShop {
    public static void main(String[] args) {
        DrinkFactory coffee = new Coffee();
        coffee.prepareDrink();
        DrinkFactory tea = new Tea();
        tea.prepareDrink();
    }
}

abstract class DrinkFactory {
    public final void prepareDrink() {
        fatchWater();
        boiledWater();
        addStuff();
        addRelish();
        pourToPot();
        if (needPackUp()) {    packUp(); }
    }
    private void fatchWater() { System.out.println("Fetching some water now... ");}
    protected void boiledWater() { System.out.println("Boiled Water now..."); }
    protected void pourToPot() { System.out.println("Pour the drink into pot...");}
    protected abstract void addRelish();
    protected abstract void addStuff();
    protected Boolean needPackUp() { return false; }
    protected void packUp() { System.out.println("Packing now...");}
}
class Coffee extends DrinkFactory {
    @Override
    protected void boiledWater() { System.out.println("Boiled water to 100 degree..."); }
    @Override
    protected void addRelish() {System.out.println("Adding milk...");}
    @Override
    protected void addStuff() {System.out.println("Adding coffee...");}
    @Override
    protected Boolean needPackUp() { return true; }
}
class Tea extends DrinkFactory {
    @Override protected void addRelish() {
        BufferedReader reader = new BufferedReader(new InputStreamReader(System.in));
        System.out.println("Do you want to add some juice? y/n");
        try {
            String ans = reader.readLine();
            if (ans.toLowerCase().equals("y")) {
                System.out.println("Adding juice now...");
            } else throw new RuntimeException("I hate you");
        } catch (Exception e) {
            System.out.println("Ok, I will skip this step");
        } finally {
            System.out.println("Adding process done!");
        }
    }
    @Override protected void addStuff() {System.out.println("Adding tea leaf...");}
}
```

以上代码展示了咖啡和茶的制作过程，在其中遵循了“好莱坞原则”：在较低层次的类中不应该调用较高（较为抽象）层次的方法，而是应该在较为抽象的方法中调用较低层次的方法（通过继承隐式调用，通过RTTI、多态进行调用）。注意，此处较高层次调用较低层次不是说要在较高层次内部保留较低层次的类型，而是说，在较高层次调用抽像接口的时候，实际通过转型或者继承动态的调用了较低层次类的实现。至于较低层次不应该调用较高层次，这是指的是，你不应该在一个具体的类中保存任何抽像的类并调用它们任何的代码。而是应该提供一个静态的实现了抽像接口的方法，以供较高层次在它处通过构造具体类的方式进行调用。

注意到，在上述代码中使用了 protected 以允许子类提供自己的实现，使用了 private 方法以保证饮料的水源健康可靠。注意到我们是如何使用钩子让 addRelish 方法是在算法中进入别的分支，提供其他服务的。这里的“外带”不应该放入主分支，然后提供null，这样不合情理，外带是一个可选选项，而不是一个默认为空的必选项。注意，对于算法逻辑，使用 final 关键字保证子类可见但不可修改。

## OO原则（好莱坞原则）：别打给我，我会找你

定义：**高层次的类通过抽像接口构造具体示例以调用低层次的类以提供服务，但是低层次的类不能调用高层次的类。换句话说，由超类主控一切，当它们需要的时候，自然会去调用子类。**在子类中，不要调用超类，因为一旦超类和子类互相调用形成闭环，那么代码将难以修改（势必会互相影响）。

## 提供服务* vs 接口标准化

和上一章不同，这一章的这些方法更偏向“提供服务”，而不是“实现接口的标准化”。为了避免提供服务时的耦合，我们需要遵循在超类中定义服务算法，在子类中提供算法步骤的具体实现的方式，由超类调用子类完成不同的服务。

提供服务分为两大类型：交给子类，算法拆分，使用继承复用代码。或者使用组合，将算法合并成策略，设置为状态，运行时可以更改，更有弹性。模板方法和工厂方法都是选择了使用继承，交给子类，而策略模式则更强调策略、整合和组合。

## 交给子类 OR 使用组合：策略，模板和工厂

模板方法模式和工厂方法模式是典型的“交给子类”。通过继承，工厂方法的子类定义了对象，使用由抽象超类提供的服务算法完成服务。而模板方法的子类并没有提供对象，而是提供了算法的一个部分，然后使用由抽象超类提供的服务算法完成服务。

策略模式也是为了“提供服务”，不过和“交给子类不同”，其采用的是组合的方式，将互相替代的算法通过外部传递并且保存成具体实例，然后调用这个实例完成服务。策略模式的算法更适合互相替代的算法，而不是庞大的，需要切分并且每个步骤不同的算法（比如选择一种加密方式，这些加密方式互相替代，使用策略模式更合适。再比如客户端连接服务器，需要经过一系列的层：物理层、链路层、网际层、应用层是顺序结构，但是各个层可选的标准不同，这就更适合模板方法）。

当然，策略模式和模板方法模式实际更为相似。我们可以为定义几组不同的连接方式，比如策略A，物理层使用以太网，链路层使用IP，网际层使用TCP，应用层选择HTTP。策略B，物理层使用WLAN，链路层使用IP，网际层使用TCP，应用层使用HTTP。可以将算法拆分，交给子类使用继承，也可以将算法合并成几种情况，形成策略，然后使用组合。

策略模式可以在运行时更改策略，灵活性更好。依赖程度更低。但是模板方法模式重用了很多代码，效率更高，对于庞大的算法，分层的结构更清晰。而策略模式面对庞大的算法可能有大量的不同策略，略显僵硬。